# Figures for FluidSF paper

In [1]:
import matplotlib_inline.backend_inline
import seaborn as sns

sns.set_style(style="white")
sns.set_context("talk")

matplotlib_inline.backend_inline.set_matplotlib_formats("png", dpi=200)

## Figure 1: Structure functions from a 2D simulation

In [3]:
import h5py

f = h5py.File('../../docs/source/example_data/2layer_128.jld2', 'r')
grid = f['grid']
snapshots = f['snapshots']

# Initialize the grid of x and y coordinates
x = grid['x'][()]
y = grid['y'][()]

# Grab the top layer and final snapshot of the simulation for u, v, and q
u = snapshots['u']['20050'][0]
v = snapshots['v']['20050'][0]
q = snapshots['q']['20050'][0]

In [5]:
import fluidsf

sf = fluidsf.generate_structure_functions(u=u, v=v, x=x, y=y, scalar=q, sf_type=["ASF_V", "ASF_S", "LLL", "LSS"])

ValueError: too many values to unpack (expected 2)